# Mounting Google Drive

In [0]:
##### before running it, make sure you don't have lots of big files in your google drive
##### otherwise it's going to take too long to finish running it before giving the TIMEOUT error
##### also save the train_controls, train_patients, val_controls, val_patients to your drive and
##### create a "train" folder with train_controls, train_patients in it, and 
##### a "val" folder with val_controls, val_patients in it.
##### change the train_dir and val_dir in the next cell to the dir of your train and val folder
##### e.g. my train and val folder are saved in the "deep learning" dir

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Loading Train, Val, and Test Data Using Keras ImageDataGenerator

In [0]:
train_dir = "/gdrive/My Drive/trainProbMaps/"
val_dir = "/gdrive/My Drive/valProbMaps/"
test_dir = "/gdrive/My Drive/testProbMaps/"

img_width = 224
img_height = 224
batch_size = 1
channels = 3
epochs = 50
nb_train_samples = 408
nb_valid_samples = 149
nb_test_samples = 192

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)             
valid_datagen = ImageDataGenerator(rescale=1./255)    
test_datagen = ImageDataGenerator(rescale=1./255) 

#EGD data
egd_dir = "/gdrive/My Drive/EGD/"

egd_datagen = ImageDataGenerator(rescale=1./255)

egd_generator = egd_datagen.flow_from_directory(
    egd_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)   

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True) #weight toward one class or another

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

Using TensorFlow backend.


Found 102 images belonging to 2 classes.
Found 408 images belonging to 2 classes.
Found 149 images belonging to 2 classes.
Found 192 images belonging to 2 classes.


# Building the Model Architecture & Training the Model

In [0]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential

from keras.callbacks import TensorBoard

#from keras.layers import Input, Dense
from keras.models import Model

input_shape = (img_height, img_width, channels)

middle = Sequential(name="middle")

middle.add(Conv2D(32,(11, 11), input_shape=input_shape))
middle.add(Activation('relu'))
middle.add(MaxPooling2D(pool_size=(4, 4)))

middle.add(Conv2D(32,(7, 7), input_shape=input_shape))
middle.add(Activation('relu'))
middle.add(MaxPooling2D(pool_size=(4, 4)))

middle.add(Conv2D(64,(3, 3), input_shape=input_shape))
middle.add(Activation('relu'))
middle.add(MaxPooling2D(pool_size=(2, 2)))

model = Sequential()
model.add(middle)
model.add(Flatten())



model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
model.fit_generator(
    test_generator,
    steps_per_epoch=nb_train_samples / batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=nb_valid_samples / batch_size,
    shuffle=False) # // batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
middle (Sequential)          (None, 4, 4, 64)          80352     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total para

Saved model to disk


In [0]:
# for input_batch, label_batch in train_generator:
#     print(middle.predict(input_batch).shape)
#     break

(200, 26, 26, 64)


In [0]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential

from PIL import Image

Image.MAX_IMAGE_PIXELS = None

from keras import layers

from keras.callbacks import TensorBoard

#from keras.layers import Input, Dense
from keras import layers
from keras.applications import resnet50
from keras import optimizers
from keras.applications import VGG16

import keras

#img_width = 475
#img_height = 388

img_width = 224
img_height = 224

from PIL import Image

Image.MAX_IMAGE_PIXELS = None

#EGD data
egd_dir = "/gdrive/My Drive/EGD/"

egd_datagen = ImageDataGenerator(rescale=1./255)

egd_generator = egd_datagen.flow_from_directory(
    egd_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

# vgg_model = VGG16(weights='imagenet',
#                   include_top=False)
# vgg_model.trainable = False

# x = keras.layers.Flatten(name='flatten')(vgg_model)
# x = keras.layers.Dense(4096, activation='relu', name='fc1')(x)
# x = keras.layers.Dense(4096, activation='relu', name='fc2')(x)
# x = keras.layers.Dense(1000, activation='softmax', name='predictions')(x)
# new_model = keras.models.Model(inputs=inp, outputs=x)
# new_model.compile(optimizer='adam', loss='categorical_crossentropy', 
#                   metrics=['accuracy'])

# new_model.fit_generator(
#     egd_generator,
#     steps_per_epoch=102 / batch_size,
#     epochs=epochs,
#      validation_data=validation_generator,
#      validation_steps=nb_validation_samples / batch_size,
#     shuffle=True) # // batch_size)



features = np.zeros(shape=(102, 4, 4, 64))
labels = np.zeros(shape=(102))
i = 0
for inputs_batch, labels_batch in egd_generator:
    features_batch = middle.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= 102:
        break
        
print(features.shape, labels.shape)


Found 102 images belonging to 2 classes.
(102, 4, 4, 64) (102,)


In [0]:
# produce 10-fold cross validation and train the model 10 times to get the average val_accuracy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics


kf = KFold(n_splits=10)
kf.get_n_splits(features)

fold_num = 0
count = 0

score_list = []
auc_list = []

for train_index, val_index in kf.split(features):
    clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
    clf.fit(np.reshape(features[train_index], (len(train_index), 4*4*64)), labels[train_index])
    valid_prediction = clf.predict(np.reshape(features[val_index], (len(val_index), 4*4*64)))
    valid_score = clf.score(np.reshape(features[val_index], (len(val_index), 4*4*64)), labels[val_index])
    fpr, tpr, thresholds = metrics.roc_curve(labels[val_index], valid_prediction, pos_label=1)
    valid_auc = metrics.auc(fpr, tpr)
    score_list.append(valid_score)
    auc_list.append(valid_auc)
    print(labels[val_index])
    print(valid_prediction)
    
print("mean score is: {}".format(np.mean(score_list)))
print("mean AUC is: {}".format(np.mean(auc_list)))

#print(labels[val_index])
#print(valid_prediction)

[1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
[1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1.]
[1. 0. 1. 1. 0. 1. 1. 0. 0. 1.]
[1. 0. 1. 1. 0. 1. 1. 0. 1. 1.]
[1. 1. 1. 1. 0. 1. 1. 1. 0. 1.]
[0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
[1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
[1. 1. 0. 1. 1. 0. 1. 1. 0. 0.]
[1. 1. 1. 0. 1. 1. 0. 1. 0. 0.]
[1. 1. 1. 0. 1. 1. 0. 1. 0. 0.]
[1. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
[1. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
[0. 1. 1. 0. 0. 0. 1. 0. 1. 1.]
[0. 1. 0. 1. 0. 1. 1. 1. 0. 0.]
[0. 1. 0. 1. 0. 1. 1. 1. 0. 0.]
[0. 1. 1. 0. 1. 1. 1. 0. 0. 0.]
[0. 1. 0. 0. 1. 1. 1. 0. 1. 0.]
mean score is: 0.9100000000000001
mean AUC is: 0.9241666666666667


In [0]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
clf.fit(np.reshape(train_features, (len(train_features), 26*26*64)), train_labels)
valid_prediction = clf.predict(np.reshape(valid_features, (len(valid_features), 26*26*64)))
valid_score = clf.score(np.reshape(valid_features, (len(valid_features), 26*26*64)), valid_labels)
fpr, tpr, thresholds = metrics.roc_curve(valid_labels, valid_prediction, pos_label=1)
valid_auc = metrics.auc(fpr, tpr)

print("score: {}".format(valid_score))
print("AUC: {}".format(valid_auc))

score: 0.09375
AUC: nan


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [0]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
clf.fit(np.reshape(train_features, (len(train_features), 26*26*64)), train_labels)
test_prediction = clf.predict(np.reshape(test_features, (len(test_features), 26*26*64)))
test_score = clf.score(np.reshape(test_features, (len(test_features), 26*26*64)), test_labels)
fpr, tpr, thresholds = metrics.roc_curve(test_labels, test_prediction, pos_label=1)
test_auc = metrics.auc(fpr, tpr)

print("score: {}".format(test_score))
print("AUC: {}".format(test_auc))

score: 0.20557851239669422
AUC: 0.4806315104166667


In [0]:
# divide to 10-folds

# produce 10-fold cross validation and train the model 10 times to get the average val_accuracy
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

kf = KFold(n_splits=10)
kf.get_n_splits(features)

fold_num = 0
count = 0

score_list = []
auc_list = []

for train_index, val_index in kf.split(features):
    clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
    clf.fit(np.reshape(features[train_index], (len(train_index), 388*475*3)), labels[train_index])
    valid_prediction = clf.predict(np.reshape(features[val_index], (len(val_index), 388*475*3)))
    valid_score = clf.score(np.reshape(features[val_index], (len(val_index), 388*475*3)), labels[val_index])
    fpr, tpr, thresholds = metrics.roc_curve(labels[val_index], valid_prediction, pos_label=1)
    valid_auc = metrics.auc(fpr, tpr)
    score_list.append(valid_score)
    auc_list.append(valid_auc)
    
print("mean score is: {}".format(np.mean(score_list)))
print("mean AUC is: {}".format(np.mean(auc_list)))


In [0]:
# !pip install keras-vis
!pip install git+https://github.com/raghakot/keras-vis.git

from keras.applications import ResNet50
from vis.utils import utils
from keras import activations

# Hide warnings on Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')

  Cloning https://github.com/raghakot/keras-vis.git to /tmp/pip-req-build-61rk4mng
  Running setup.py bdist_wheel for keras-vis ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-q11y7kbk/wheels/c5/ae/e7/b34d1cb48b1898f606a5cce08ebc9521fa0588f37f1e590d9f
Successfully built keras-vis


# Testing on Untouched Subset of Atlantis Patients and Controls

In [0]:
#TEST Accuracy on fraction from same group results in quite high accuracy but could get higher...
#print(train_generator.filenames)
#print(validation_generator.filenames)

nb_test_samples = 200

result = model.evaluate_generator(test_generator, steps=nb_test_samples / batch_size)
print(result)

predictions = model.predict_generator(test_generator, steps=nb_test_samples / batch_size)
print(np.round(predictions))
print(len(predictions))
img_path = "/gdrive/My Drive/deep learning/Project/testProbMaps/patients/24215.png"

from keras.preprocessing import image                             
import numpy as np

img = image.load_img(img_path, target_size=(img_width, img_height))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.                                               


print(img_tensor.shape)
#validation_generator.classes

# Testing on Early Glaucoma Dataset

In [0]:
#TEST Accuracy on Early Glaucoma Dataset (actually about 50% of these are classified as controls by experts 
#but were patients according to the training group criteria), so 50% accuracy make sense!!
test_dir = "/gdrive/My Drive/test/" 

img_width = 600
img_height = 450

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

nb_test_samples = 102

result = model.evaluate_generator(test_generator, steps=nb_test_samples / batch_size)
print(result)

predictions = model.predict_generator(test_generator, steps=nb_test_samples / batch_size)
print(np.round(predictions))
print(len(predictions))



In [0]:
print(test_generator.filenames)

# Visualizing Layer Activations

In [0]:
import matplotlib.pyplot as plt

#plt.imshow(img_tensor[0])
#plt.show()

from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]               
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)

import matplotlib.pyplot as plt
plt.figure()
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

model.summary